In [1]:
import os
from PIL import Image
import xml.etree.cElementTree as ET 
import re

In [2]:
filenames = os.listdir("./")
dirs = [d for d in filenames if "-output" in d]
output_dir = "outputs/"

In [231]:
for dir in dirs:
    app_name = dir.split("-")[0]
    dir_name = os.path.join(dir, "stoat_fsm_output", "ui")
    files_names = os.listdir(dir_name)
    imgs = [d for d in files_names if "png" in d]
    for i in imgs:
        xml_name = [d for d in files_names if d == i.split(".")[0] + ".xml"]
        if len(xml_name) > 0:
            tree = ET.parse(dir_name + "/" + xml_name[0]) 
            root = tree.getroot()

            count = 0
            img_name = i
            find_all_element_by_attribute(root, "node", "class", "android.widget.Button")

In [224]:
count = 0
def mask_and_save_image(box):
    global count
    
    im = Image.open(dir_name + "/" + img_name)
    img_names = img_name.split('.')
    region=im.crop(box)

    mask = Image.new("RGBA",(box[2]-box[0],box[3]-box[1]),(0,0,0))

    im.paste(mask,box)
    im.save(output_dir + app_name + "-" +  img_names[0] + "-mask_" + str(count) + "." + img_names[1])
    count += 1

In [219]:
# Find node recursively
def find_all_element_by_attribute(node, element_name, attribute, find):
    """
    This function use preorder traversal to find the all the XML objects by its attributs value.
    Parameters:
    node        : XML object.
    element_name: Target object name (name of node).
    attribute   : Target attribut name in XML object.
    find        : Target attribute value which mean to find.
    """
    if attribute in node.attrib and node.attrib[attribute] == find:
        # Operations after find the target XML objects.
        bounds = node.attrib['bounds']
        bounds = re.findall(r'(\w*[0-9]+)\w*',bounds)
        bounds = [int(i) for i in bounds]
        mask_and_save_image(bounds)
    
    # Visit all the target objects in current object.
    for n in node.findall(element_name):
        find_all_element_by_attribute(n, element_name, attribute, find)

In [205]:
count = 0
find_all_element_by_attribute(root, "node", "class", "android.widget.Button")

In [141]:
im = Image.open(dirname + "S_692.png")
box = (666,1123,800,1216)
region=im.crop(box)
region.show()
mask = Image.new("RGBA",(box[2]-box[0],box[3]-box[1]),(0,0,0))
mask.show()
im.paste(mask,box)
im.show()